In [1]:
import pandas as pd
import xlwings as xw
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pythoncom

In [2]:
# gsheet_id = '1AUqkl6Nd9gRUooxtWiVrKUnvV_aXNQ2sAbCdvbBcxnQ'
# sheet_name = 'price'
# gsheet_url = f'https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
#
# phone_list = pd.read_csv(gsheet_url)
phone_list = pd.read_csv("stock.csv")
print(phone_list)

                        Name Capacity  Color Grade  Qty        Cost
0           Galaxy A31 A315G     64GB    NaN     C    8  140.000000
1     Galaxy A5 (2016) A510F     16GB    NaN     A   16   56.050132
2  Galaxy A5 (2017) SM-A520F     32GB    NaN     C   11  102.758576
3       Galaxy A52s 5G A528B    128GB    NaN     C    8   65.000000
4           Galaxy A6 A600FN     32GB    NaN     C   27  500.000000
5           Galaxy J6 J600FN     32GB    NaN     A    6  103.000000
6        Galaxy M11 SM-M115F     32GB    NaN     A   11  103.000000
7        Galaxy M13 SM-M135F     64GB    NaN     A    6   75.000000
8  Galaxy Note 10 Lite N770F    128GB    NaN     C   30   75.000000


In [3]:
for i, row in phone_list.iterrows():
    print(row[3])

C
A
C
C
C
A
A
A
C


In [4]:
def retrieve_data():
    # Initialize the COM subsystem
    pythoncom.CoInitialize()

    main_df = pd.DataFrame(columns=['model',
                                    # 'category',
                                    'grade',
                                    # 'wesell',
                                    'webuy_cash',
                                    # 'webuy_voucher',
                                    'low_margin %',
                                    'mid_margin %',
                                    'high_margin %'])


    grade = phone_list[['Grade']]
    for i, row in phone_list.iterrows():
        # website = f"https://uk.webuy.com/search?stext=+{row[0]}+&Grade={row[3]}+&Capacity={row[1]}&Main%2520Colour={row[2]}"
        website = f"https://uk.webuy.com/search?stext=+{row[0:2]}+&Grade={row[3]}"
        # Selenium and Chrome webscraping configs

         # headless chrome option
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/110.0"

        options = webdriver.ChromeOptions()
        # options.headless = True
        options.add_argument('--headless')
        options.add_argument(f'user-agent={user_agent}')
        options.add_argument("--window-size=1920,1080")
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--allow-running-insecure-content')
        options.add_argument("--disable-extensions")
        options.add_argument("--proxy-server='direct://'")
        options.add_argument("--proxy-bypass-list=*")
        options.add_argument("--start-maximized")
        options.add_argument('--disable-gpu')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--no-sandbox')
        # driver = webdriver.Chrome(executable_path="chromedriver.exe", options=options)
        driver = webdriver.Chrome(options=options)
        driver.get(website)

        # Wait for the cookie acceptance button to be visible
        wait = WebDriverWait(driver, 10)
        cookie_button = wait.until(EC.visibility_of_element_located((By.ID, "onetrust-accept-btn-handler")))

        # Click the acceptance button
        cookie_button.click()

        # Extracting the data from the website
        phone_models = driver.find_elements(By.XPATH, "//div[@class='desc']")

        # List to append the extracted data
        model = []
        # device = []
        grade = []
        # wesell = []
        webuy_cash = []
        # webuy_voucher = []
        low_margin = []
        mid_margin = []
        high_margin = []

        # Looping through each phone search results to get the values
        for models in phone_models:
            name = models.find_element(By.XPATH, ".//span[@class='ais-Highlight']").text
            model.append(name)
            # device.append(models.find_element(By.XPATH, "./p").text)
            grade.append(str(name)[-1])
            # wesell.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeSell for')]").text)
            webuy_cash.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeBuy for cash')]").text)
            # webuy_voucher.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeBuy for voucher')]").text)
            low_margin.append(0.20)
            mid_margin.append(0.25)
            high_margin.append(0.30)

        # Rearranging the data into a data frame
        df = pd.DataFrame({'model': model,
                           # 'category': device,
                           'grade': grade,
                           # 'wesell': wesell,
                           'webuy_cash': webuy_cash,
                           # 'webuy_voucher': webuy_voucher,
                           'low_margin %': low_margin,
                           'mid_margin %': mid_margin,
                           'high_margin %': high_margin})

        first_row = df[:1]

        main_df = main_df.append(first_row, ignore_index=True)

    return main_df

In [5]:
retrieve_data()

C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(first_row, ignore_index=True)
C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(first_row, ignore_index=True)
C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(first_row, ignore_index=True)
C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.appen

,model,grade,webuy_cash,low_margin %,mid_margin %,high_margin %
0,"Samsung Galaxy Tab A T280 7.0"" (2016) 8GB Blac...",C,WeBuy for cash £16.00,0.2,0.25,0.3
1,"Samsung Galaxy A5 A510F (2016) 16GB Black, O2 A",A,WeBuy for cash £37.00,0.2,0.25,0.3
2,"Samsung Galaxy A5 A520F (2017) 32GB Black, Unl...",C,WeBuy for cash £39.00,0.2,0.25,0.3
3,Samsung Galaxy A52s 5G Dual Sim (6GB+128GB) Aw...,C,WeBuy for cash £110.00,0.2,0.25,0.3
4,"Samsung Galaxy A6 (2018) 32GB Black, Unlocked C",C,WeBuy for cash £43.00,0.2,0.25,0.3
5,"Samsung Galaxy SM-T290 Tab A 8.0"" (2019) 32GB ...",A,WeBuy for cash £42.00,0.2,0.25,0.3
6,"Samsung Galaxy Tab 3 SM-T211 7"" 8GB, Unlocked A",A,WeBuy for cash £19.00,0.2,0.25,0.3
7,Samsung Galaxy Note 10 Lite Dual Sim 128GB Aur...,C,WeBuy for cash £101.00,0.2,0.25,0.3


In [11]:
def rearrange_data():

    final_df = pd.concat([phone_list, retrieve_data()], axis=1)

    # Converting text into numbers and calculations
    # final_df['wesell'] = final_df['wesell'].str.replace('WeSell for £', '').astype(float)
    final_df['webuy_cash'] = final_df['webuy_cash'].str.replace('WeBuy for cash £', '').astype(float)
    # final_df['webuy_voucher'] = final_df['webuy_voucher'].str.replace('WeBuy for voucher £', '').astype(float)
    final_df['low_margin_cost'] = final_df['webuy_cash'] * 0.80
    final_df['mid_margin_cost'] = final_df['webuy_cash'] * 0.75
    final_df['high_margin_cost'] = final_df['webuy_cash'] * 0.70
    final_df['low_margin %'] = final_df['low_margin %'].map('{:.1%}'.format)
    final_df['mid_margin %'] = final_df['mid_margin %'].map('{:.1%}'.format)
    final_df['high_margin %'] = final_df['high_margin %'].map('{:.1%}'.format)

    # Rename columns
    final_df = final_df.rename(columns={'Name': 'Supplier_Name',
                                        'Qty': 'Supplier_Qty',
                                        'Cost': 'Supplier_Cost'})
    final_df = final_df[[
            'Supplier_Name',
            'Capacity',
            'Color',
            'Grade',
            'Supplier_Qty',
            'Supplier_Cost',
            'model',
            'grade',
            'webuy_cash',
            'low_margin %',
            'low_margin_cost',
            'mid_margin %',
            'mid_margin_cost',
            'high_margin %',
            'high_margin_cost'
    ]]
    # Close the web browser and return
    # driver.quit()

    return final_df

In [12]:
rearrange_data()

C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(first_row, ignore_index=True)
C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(first_row, ignore_index=True)
C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(first_row, ignore_index=True)
C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.appen

,Supplier_Name,Capacity,Color,Grade,Supplier_Qty,Supplier_Cost,model,grade,webuy_cash,low_margin %,low_margin_cost,mid_margin %,mid_margin_cost,high_margin %,high_margin_cost
0,Galaxy A31 A315G,64GB,NaN,C,8,140.000000,"Samsung Galaxy Tab A T280 7.0"" (2016) 8GB Blac...",C,16.0,20.0%,12.8,25.0%,12.00,30.0%,11.2
1,Galaxy A5 (2016) A510F,16GB,NaN,A,16,56.050132,"Samsung Galaxy A5 A510F (2016) 16GB Black, O2 A",A,37.0,20.0%,29.6,25.0%,27.75,30.0%,25.9
2,Galaxy A5 (2017) SM-A520F,32GB,NaN,C,11,102.758576,"Samsung Galaxy A5 A520F (2017) 32GB Black, Unl...",C,39.0,20.0%,31.2,25.0%,29.25,30.0%,27.3
3,Galaxy A52s 5G A528B,128GB,NaN,C,8,65.000000,Samsung Galaxy A52s 5G Dual Sim (6GB+128GB) Aw...,C,110.0,20.0%,88.0,25.0%,82.50,30.0%,77.0
4,Galaxy A6 A600FN,32GB,NaN,C,27,500.000000,"Samsung Galaxy A6 (2018) 32GB Black, Unlocked C",C,43.0,20.0%,34.4,25.0%,32.25,30.0%,30.1
5,Galaxy J6 J600FN,32GB,NaN,A,6,103.000000,"Samsung Galaxy Xcover 5 Dual Sim 64GB Black, V...",A,57.0,20.0%,45.6,25.0%,42.75,30.0%,39.9
6,Galaxy M11 SM-M115F,32GB,NaN,A,11,103.000000,"Samsung Galaxy SM-T290 Tab A 8.0"" (2019) 32GB ...",A,42.0,20.0%,33.6,25.0%,31.50,30.0%,29.4
7,Galaxy M13 SM-M135F,64GB,NaN,A,6,75.000000,"Samsung Galaxy Tab 3 SM-T211 7"" 8GB, Unlocked A",A,19.0,20.0%,15.2,25.0%,14.25,30.0%,13.3
8,Galaxy Note 10 Lite N770F,128GB,NaN,C,30,75.000000,Samsung Galaxy Note 10 Lite Dual Sim 128GB Aur...,C,101.0,20.0%,80.8,25.0%,75.75,30.0%,70.7


In [ ]:
def save_to_excel():
    with xw.App(visible=False) as app:
        wb = app.books.open('Template.xlsx')
        wb.sheets['Sheet1'].range('A3').value = rearrange_data()
        wb.sheets['Sheet1'].range('3:3').delete()
        wb.save("CEX_Product_Price.xlsx")

In [8]:
def save_to_excel():
    with xw.App(visible=False) as app:
        wb = app.books.open('Template.xlsx')
        wb.sheets['Sheet1'].range('A3').value = rearrange_data()
        wb.sheets['Sheet1'].range('3:3').delete()
        wb.save("CEX_Product_Price.xlsx")

In [10]:
def save_to_excel():
    with xw.App(visible=False) as app:
        wb = app.books.open('Template.xlsx')
        wb.sheets['Sheet1'].range('A3').value = rearrange_data()
        wb.sheets['Sheet1'].range('3:3').delete()
        wb.save("CEX_Product_Price.xlsx")

C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(first_row, ignore_index=True)
C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(first_row, ignore_index=True)
C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(first_row, ignore_index=True)
C:\Users\mrmra\AppData\Local\Temp\ipykernel_25100\3301846072.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.appen

In [ ]:
save_to_excel()

<h4>Tkinter GUI</h4>

In [3]:
class CexBot:

    def __init__(self):

        self.root = tk.Tk()

        self.Label = tk.Label(self.root, text="CEX Price Updater", font=('Arial', 18))
        self.Label.pack(pady=10, padx=10)

        # input file path variable
        self.label_file_path = tk.Label(self.root, text='')

        self.select_button = tk.Button(self.root, text="Select File", font=("Arial", 18),
                                       command=self.select_file, bg='lightblue')
        self.select_button.pack(padx=10, pady=10)

        # output file save location variable
        self.output_save_location = tk.Label(self.root, text='')

        self.update_button = tk.Button(self.root, text="Update & Save", font=("Arial", 18),
                                       command=self.update_and_save, bg='lightblue')
        self.update_button.pack(padx=10, pady=10)

        # Progress bar
        self.my_progress = ttk.Progressbar(self.root, orient='horizontal',
                                           length=300, mode='indeterminate')
        self.my_progress.pack(pady=20)

        self.root.mainloop()

    def select_file(self):
        file_path = filedialog.askopenfilename(title='Select Input File',
                                               filetypes=(("xlsx files", "*.xlsx"), ("All Files", "*.*"))
                                               )
        self.label_file_path["text"] = file_path
        return None

    def update_and_save(self):

        # progress bar starting
        self.my_progress.start(10)
        self.root.update_idletasks()

        # Save location
        save_folder = filedialog.askdirectory(title="Select a folder to Save")
        self.output_save_location["text"] = save_folder
        save_location = self.output_save_location["text"]
        save_folder_location = r"{}".format(save_location)

        # Read the phone model list to get the price
        excel_file_path = self.label_file_path["text"]

        try:
            # excel_file_name = r"{}".format(excel_file_path)
            # phone_list = pd.read_csv(excel_file_name)
            phone_list = pd.read_csv("stock.csv")

        except ValueError:
            tk.messagebox.showerror("Information", "The file you have entered is invalid")
            self.my_progress.stop()
            return None
        except FileNotFoundError:
            tk.messagebox.showerror("Information", f"No such file as {excel_file_path}")
            self.my_progress.stop()
            return None

        def retrieve_data():
            # Initialize the COM subsystem
            pythoncom.CoInitialize()

            main_df = pd.DataFrame(columns=['model',
                                            'category',
                                            'grade',
                                            'wesell',
                                            'webuy_cash',
                                            'webuy_voucher',
                                            'low_margin %',
                                            'mid_margin %',
                                            'high_margin %'])

            for i, row in phone_list.iterrows():
                website = f"https://uk.webuy.com/search?stext=+{row[0]}&Grade=C"

                # Selenium and Chrome webscraping configs
                # options = webdriver.ChromeOptions()
                # chrome_options = ChromeOptions()
                # chrome_options.add_argument('--headless')
                # chrome_options.add_argument('--disable-gpu')
                # options.add_experimental_option("detach", False)
                # driver = webdriver.Chrome(options=options)
                # driver.get(website)

                # headless chrome option
                user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/110.0"

                options = webdriver.ChromeOptions()
                # options.headless = True
                options.add_argument('--headless')
                options.add_argument(f'user-agent={user_agent}')
                options.add_argument("--window-size=1920,1080")
                options.add_argument('--ignore-certificate-errors')
                options.add_argument('--allow-running-insecure-content')
                options.add_argument("--disable-extensions")
                options.add_argument("--proxy-server='direct://'")
                options.add_argument("--proxy-bypass-list=*")
                options.add_argument("--start-maximized")
                options.add_argument('--disable-gpu')
                options.add_argument('--disable-dev-shm-usage')
                options.add_argument('--no-sandbox')
                # driver = webdriver.Chrome(executable_path="chromedriver.exe", options=options)
                driver = webdriver.Chrome(options=options)
                driver.get(website)

                # Wait for the cookie acceptance button to be visible
                wait = WebDriverWait(driver, 10)
                cookie_button = wait.until(EC.visibility_of_element_located((By.ID, "onetrust-accept-btn-handler")))

                # Click the acceptance button
                cookie_button.click()

                # Extracting the data from the website
                phone_models = driver.find_elements(By.XPATH, "//div[@class='desc']")

                # List to append the extracted data
                model = []
                device = []
                grade = []
                wesell = []
                webuy_cash = []
                webuy_voucher = []
                low_margin = []
                mid_margin = []
                high_margin = []

                # Looping through each phone search results to get the values
                for models in phone_models:
                    model.append(models.find_element(By.XPATH, ".//span[@class='ais-Highlight']").text)
                    device.append(models.find_element(By.XPATH, "./p").text)
                    grade.append('C')
                    wesell.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeSell for')]").text)
                    webuy_cash.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeBuy for cash')]").text)
                    webuy_voucher.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeBuy for voucher')]").text)
                    low_margin.append(0.20)
                    mid_margin.append(0.25)
                    high_margin.append(0.30)

                # Rearranging the data into a data frame
                df = pd.DataFrame({'model': model,
                                   'category': device,
                                   'grade': grade,
                                   'wesell': wesell,
                                   'webuy_cash': webuy_cash,
                                   'webuy_voucher': webuy_voucher,
                                   'low_margin %': low_margin,
                                   'mid_margin %': mid_margin,
                                   'high_margin %': high_margin})

                first_row = df[:1]

                main_df = main_df.append(first_row, ignore_index=True)

            def rearrange_data():

                final_df = pd.concat([phone_list, main_df], axis=1)

                # Converting text into numbers and calculations
                final_df['wesell'] = final_df['wesell'].str.replace('WeSell for £', '').astype(float)
                final_df['webuy_cash'] = final_df['webuy_cash'].str.replace('WeBuy for cash £', '').astype(float)
                final_df['webuy_voucher'] = final_df['webuy_voucher'].str.replace('WeBuy for voucher £', '').astype(float)
                final_df['low_margin_cost'] = final_df['webuy_cash'] * 0.80
                final_df['mid_margin_cost'] = final_df['webuy_cash'] * 0.75
                final_df['high_margin_cost'] = final_df['webuy_cash'] * 0.70
                final_df['low_margin %'] = final_df['low_margin %'].map('{:.1%}'.format)
                final_df['mid_margin %'] = final_df['mid_margin %'].map('{:.1%}'.format)
                final_df['high_margin %'] = final_df['high_margin %'].map('{:.1%}'.format)

                # Rename columns
                final_df = final_df.rename(columns={'Name': 'Supplier_Name',
                                                    'Qty': 'Supplier_Qty',
                                                    'Cost': 'Supplier_Cost'})
                # Close the web browser and return
                driver.quit()
                return final_df

            # export the file into preformatted excel
            def save_to_excel():
                with xw.App(visible=False) as app:
                    wb = app.books.open('Template.xlsx')
                    wb.sheets['Sheet1'].range('A3').value = rearrange_data()
                    wb.sheets['Sheet1'].range('3:3').delete()
                    wb.save(f"{save_folder_location}/CEX_Product_Price.xlsx")

            try:
                retrieve_data()
                rearrange_data()
                save_to_excel()
                self.my_progress.stop()
            except:
                tk.messagebox.showerror("Information", "Error occured during Excel conversion!")
                self.my_progress.stop()
                return None

            tk.messagebox.showinfo("Information", 'Updated & File Saved Successfully!')

            # Uninitialize the COM subsystem
            pythoncom.CoUninitialize()


In [ ]:
CexBot()


In [3]:
def retrieve_data():
    # Initialize the COM subsystem
    pythoncom.CoInitialize()

    main_df = pd.DataFrame(columns=['model',
                                    'category',
                                    'grade',
                                    'wesell',
                                    'webuy_cash',
                                    'webuy_voucher',
                                    'low_margin %',
                                    'mid_margin %',
                                    'high_margin %'])



    for i, row in phone_list.iterrows():
        website = f"https://uk.webuy.com/search?stext=+{row[0]}&Grade=C"
        
        # Selenium and Chrome webscraping configs

         # headless chrome option
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/110.0"

        options = webdriver.ChromeOptions()
        # options.headless = True
        options.add_argument('--headless')
        options.add_argument(f'user-agent={user_agent}')
        options.add_argument("--window-size=1920,1080")
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--allow-running-insecure-content')
        options.add_argument("--disable-extensions")
        options.add_argument("--proxy-server='direct://'")
        options.add_argument("--proxy-bypass-list=*")
        options.add_argument("--start-maximized")
        options.add_argument('--disable-gpu')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--no-sandbox')
        # driver = webdriver.Chrome(executable_path="chromedriver.exe", options=options)
        driver = webdriver.Chrome(options=options)
        driver.get(website)

        # Wait for the cookie acceptance button to be visible
        wait = WebDriverWait(driver, 10)
        cookie_button = wait.until(EC.visibility_of_element_located((By.ID, "onetrust-accept-btn-handler")))

        # Click the acceptance button
        cookie_button.click()

        # Extracting the data from the website
        phone_models = driver.find_elements(By.XPATH, "//div[@class='desc']")

        # List to append the extracted data
        model = []
        device = []
        grade = []
        wesell = []
        webuy_cash = []
        webuy_voucher = []
        low_margin = []
        mid_margin = []
        high_margin = []

        # Looping through each phone search results to get the values
        for models in phone_models:
            model.append(models.find_element(By.XPATH, ".//span[@class='ais-Highlight']").text)
            device.append(models.find_element(By.XPATH, "./p").text)
            grade.append('C')
            wesell.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeSell for')]").text)
            webuy_cash.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeBuy for cash')]").text)
            webuy_voucher.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeBuy for voucher')]").text)
            low_margin.append(0.20)
            mid_margin.append(0.25)
            high_margin.append(0.30)

        # Rearranging the data into a data frame
        df = pd.DataFrame({'model': model,
                           'category': device,
                           'grade': grade,
                           'wesell': wesell,
                           'webuy_cash': webuy_cash,
                           'webuy_voucher': webuy_voucher,
                           'low_margin %': low_margin,
                           'mid_margin %': mid_margin,
                           'high_margin %': high_margin})

        first_row = df[:1]

        main_df = main_df.append(first_row, ignore_index=True)
        
    return main_df

In [ ]:
retrieve_data()